In [1]:
%matplotlib inline

In [2]:
import numpy as np 
import pandas 
from itertools import product, combinations
from primer3 import calcHomodimer, calcHeterodimer, calcTm

In [3]:
# generate all possible 8-mer DNA sequences without homopolymers
# we want to avoid homopolyers in our barcodes 

In [4]:
alphabet = 'ATCG'

In [5]:
def has_homopolymers( sequence ):
    for i in range( len( sequence ) - 1 ):
        if sequence[ i ] == sequence[ i + 1 ]:
            return True 
    return False

In [6]:
barcode_length = 12 
barcodes = []
for combo in product( alphabet, repeat=barcode_length ):
    if not has_homopolymers( combo ):
        barcodes.append( ''.join( combo ) )

In [7]:
# primer sequences for our test run are 
# T7 (forward): 5'-TAATACGACTCACTATAGGG-3'
# T7term (reverse): 5'-GCTAGTTATTGCTCAGCGG-3'
# we will add barcodes to the 5' end of both primers 
# assume the Tm of the T7 primers is good as is

In [8]:
# calculate the propensity of the primers to dimerize 

In [9]:
# so this works for generic Sanger primer, 
# we make these variables 

In [10]:
forward_sanger_primer = 'TAATACGACTCACTATAGGG'
reverse_sanger_primer = 'GCTAGTTATTGCTCAGCGG'

In [11]:
# describe the input primers 

forward_seq_primer_tm = calcTm( forward_sanger_primer ) 
reverse_seq_primer_tm = calcTm( reverse_sanger_primer ) 

print( '{0:2.2f}, {1:2.2f}'.format( forward_seq_primer_tm, reverse_seq_primer_tm ) )

wt_tm = { 'forward': forward_seq_primer_tm, 'reverse': reverse_seq_primer_tm } 

44.59, 50.61


In [12]:
fwd_primers = [ 'GC' + b + forward_sanger_primer for b in barcodes ]
rev_primers = [ 'GC' + b + reverse_sanger_primer for b in barcodes ]

In [13]:
# let's make a DataFrame out of this right now 
data = [ ( fwd_primer, 'forward' ) for fwd_primer in fwd_primers ] + [ ( rev_primer, 'reverse' ) for rev_primer in rev_primers ] 
df = pandas.DataFrame( data, columns=['sequence', 'direction'] )
df.sample( 2 ) 

,sequence,direction
1075155,GCCATCGTGTCTGAGCTAGTTATTGCTCAGCGG,reverse
1381380,GCGCTACTGCATGAGCTAGTTATTGCTCAGCGG,reverse


In [14]:
# calc Tm for each 

df[ 'primary_tm' ] = list( map( lambda x: wt_tm[x], df.direction ) ) 
df[ 'secondary_tm' ] = list( map( calcTm, df.sequence ) ) 
df.sample( 2 ) 

,sequence,direction,primary_tm,secondary_tm
281886,GCTCGAGCGATAGATAATACGACTCACTATAGGG,forward,44.591869,60.621314
24993,GCATCAGTAGTGATTAATACGACTCACTATAGGG,forward,44.591869,58.163780


In [15]:
# # exclude homodimers 

# df[ 'homodimer' ] = list( map( lambda x: calcHomodimer( x ).structure_found, df.sequence ) )
# df.sample( 2 ) 

# # fwd_primers = list( filter( lambda x: calcHomodimer( x ).structure_found, fwd_primers ) ) 
# # rev_primers = list( filter( lambda x: calcHomodimer( x ).structure_found, rev_primers ) ) 
# # print( len( fwd_primers) )
# # print( len( rev_primers) )
# # I guess there are none 

In [16]:
# so, now let's filter this list down 

# first, let's exclude the homopolymers we accidentally introduced 
# that arise because the barcode is next to the sequencing primer 
# remove these homopolymers 

# second, calculate the secondary Tm (Tm of entire barcode+primer pair) 
# throw out if the secondary Tm is too far from the original Tm
# is there a way to set up the PCR that will work for a wide variety of Tm? 

# third, of all of the primers that pass, form a pool and calculate the propensity 
# of hetreodimeriztion pairwise until they are pairwise independent

# only then think about Hamming distances
# I'm guessing 100 

In [17]:
def check_introduced_homopolymers( sequence, barcode_length ):
    # barcode is on the 5' end of the sequence
    if sequence[ barcode_length ] == sequence[ barcode_length - 1 ]:
        return True 
    else:
        return False 
    
df[ 'introduced_homopolymers' ] = df.sequence.map( lambda x: check_introduced_homopolymers( x, barcode_length + 2 ) )
df.sample( 2 ) 

,sequence,direction,primary_tm,secondary_tm,introduced_homopolymers
1345536,GCGTGTATGACGATGCTAGTTATTGCTCAGCGG,reverse,50.612736,64.254941,False
1326391,GCGTCTACTCACGTGCTAGTTATTGCTCAGCGG,reverse,50.612736,65.250066,False


In [18]:
df.sample( 10 ) 

,sequence,direction,primary_tm,secondary_tm,introduced_homopolymers
972501,GCTCTCAGATACGAGCTAGTTATTGCTCAGCGG,reverse,50.612736,63.206257,False
307788,GCTGACGCACGTGATAATACGACTCACTATAGGG,forward,44.591869,62.948908,False
27804,GCATCTAGACAGTATAATACGACTCACTATAGGG,forward,44.591869,57.487200,False
989585,GCTCGAGTCTATGCGCTAGTTATTGCTCAGCGG,reverse,50.612736,65.178618,False
1333835,GCGTCGAGCATAGCGCTAGTTATTGCTCAGCGG,reverse,50.612736,67.109905,False
106422,GCACGTACGCGTGATAATACGACTCACTATAGGG,forward,44.591869,62.972697,False
1146216,GCCTCAGATGCTCAGCTAGTTATTGCTCAGCGG,reverse,50.612736,65.168800,False
1314003,GCGTAGAGTCAGTAGCTAGTTATTGCTCAGCGG,reverse,50.612736,63.159162,False
521848,GCCGCTCTGTCGACTAATACGACTCACTATAGGG,forward,44.591869,63.661757,False
1234978,GCCGCGAGATCGCTGCTAGTTATTGCTCAGCGG,reverse,50.612736,68.531299,False


In [19]:
def hamming_distance( u, v ): 
    same = 0 
    if len( u ) == len( v ):
        for i, j in zip( u, v ):
            if i == j:
                same += 1 
        return same / len( u ) 
    
hamming_distance( 
    'GCGAGATGCTACGCTAATACGACTCACTATAGGG',
    'GCGTGTCAGTCAGTTAATACGACTCACTATAGGG', 
) 

0.7647058823529411

In [23]:
%load_ext Cython

In [24]:
%%cython 

cdef chamming_distance( u, v ):
    same = 0 
    if len( u ) == len( v ):
        for i, j in zip( u, v ):
            if i == j:
                same += 1 
        return same / len( u ) 
    
def cyhamming_distance( u, v ):
    return chamming_distance( u, v ) 

In [32]:
%%timeit 

hamming_distance( 
    'GCGAGATGCTACGCTAATACGACTCACTATAGGG',
    'GCGTGTCAGTCAGTTAATACGACTCACTATAGGG', 
) 

100000 loops, best of 3: 5.61 µs per loop


In [33]:
%%timeit 

cyhamming_distance( 
    'GCGAGATGCTACGCTAATACGACTCACTATAGGG',
    'GCGTGTCAGTCAGTTAATACGACTCACTATAGGG', 
) 

The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.86 µs per loop


In [ ]:
for i, j in combinations( df.